In [1]:
import os
import re

from constants import SOURCE_DIR

In [2]:
def rename_wav_files(directory):
    pattern = re.compile(r'(\d{1,2})-(\d{1,2})\s.*\.wav$', re.IGNORECASE)

    for filename in os.listdir(directory):
        if filename.lower().endswith('.wav'):
            match = pattern.match(filename)
            if match:
                day1 = int(match.group(1))
                day2 = int(match.group(2))
                new_filename = f"Zero Hour {day1:02d}-{day2:02d}.wav"
                src = os.path.join(directory, filename)
                dst = os.path.join(directory, new_filename)
                print(f"Renaming '{filename}' -> '{new_filename}'")
                os.rename(src, dst)

In [3]:
rename_wav_files(SOURCE_DIR)

Renaming '1-10 Chapter 05.wav' -> 'Zero Hour 01-10.wav'
Renaming '1-05 Chapter 02a.wav' -> 'Zero Hour 01-05.wav'
Renaming '1-07 Chapter 03.wav' -> 'Zero Hour 01-07.wav'
Renaming '1-02 Prologue a.wav' -> 'Zero Hour 01-02.wav'
Renaming '1-04 Chapter 01a.wav' -> 'Zero Hour 01-04.wav'
Renaming '1-08 Chapter 04a.wav' -> 'Zero Hour 01-08.wav'
Renaming '1-06 Chapter 02b.wav' -> 'Zero Hour 01-06.wav'
Renaming '1-03 Prologue b.wav' -> 'Zero Hour 01-03.wav'
Renaming '1-01 Tom Clancy Zero Hour ~ Intro.wav' -> 'Zero Hour 01-01.wav'
Renaming '1-09 Chapter 04b.wav' -> 'Zero Hour 01-09.wav'
